In [2]:
pip install requests beautifulsoup4 pandas openpyxl pykrx lxml

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 크롤러

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from pykrx import stock
from urllib.parse import quote
from datetime import datetime

# pandas 출력 설정
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 200)

# 사용자 입력 받기
input_name = input("크롤링할 종목명을 입력하세요 (예: 카카오): ").strip()
start_date_input = input("시작일을 입력하세요 (예: 2023-07-01): ").strip()
end_date_input = input("종료일을 입력하세요 (예: 2025-07-25): ").strip()

# 날짜 형식 변환
start_date = datetime.strptime(start_date_input, "%Y-%m-%d")
end_date = datetime.strptime(end_date_input, "%Y-%m-%d")

# 종목명 → 종목코드 변환
ticker_list = stock.get_market_ticker_list(market="ALL")
ticker_name_map = {stock.get_market_ticker_name(code): code for code in ticker_list}

if input_name not in ticker_name_map:
    print(f"[오류] '{input_name}'에 해당하는 종목코드를 찾을 수 없습니다.")
else:
    item_code = ticker_name_map[input_name]
    base_url = "https://finance.naver.com/research/"
    headers = {"User-Agent": "Mozilla/5.0"}

    data = []

    for i in range(1, 7):  # 페이지 수 조정 가능
        url = f"https://finance.naver.com/research/company_list.naver?searchType=itemCode&itemName={quote(input_name)}&itemCode={item_code}&page={i}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        rows = soup.select("table.type_1 tr")[2:]

        for row in rows:
            cols = row.find_all("td")
            if len(cols) < 5:
                continue

            a_tag = cols[1].find("a", href=True)
            if not a_tag or not a_tag["href"].startswith("company_read.naver"):
                continue

            title = a_tag.text.strip()
            detail_url = base_url + a_tag["href"]
            broker = cols[2].text.strip()
            pdf_tag = cols[3].find("a", href=True)
            pdf_link = pdf_tag["href"] if pdf_tag and pdf_tag["href"].endswith(".pdf") else None
            date_str = cols[4].text.strip()

            # 작성일 필터링
            try:
                date = datetime.strptime(date_str, "%y.%m.%d")
            except:
                continue

            if not (start_date <= date <= end_date):
                continue

            # 상세 페이지 크롤링 (목표가, 투자의견)
            try:
                detail_res = requests.get(detail_url, headers=headers)
                detail_soup = BeautifulSoup(detail_res.text, "html.parser")

                target_price = detail_soup.select_one("em.money")
                investment_opinion = detail_soup.select_one("em.coment")

                target_price_text = target_price.text.strip() if target_price else None
                opinion_text = investment_opinion.text.strip() if investment_opinion else None
            except:
                target_price_text = None
                opinion_text = None

            data.append({
                "제목": title,
                "상세페이지링크": detail_url,
                "증권사": broker,
                "PDF 링크": pdf_link,
                "작성일": date_str,
                "목표가": target_price_text,
                "투자의견": opinion_text
            })

            time.sleep(0.3)

    # DataFrame 생성 (링크 있는 버전)
    df_link = pd.DataFrame(data)

    # Excel 저장 (링크 있는 버전)
    filename = f"{input_name}_{start_date_input}~{end_date_input}_리포트 수집.xlsx"
    df_link.to_excel(filename, index=False, engine='openpyxl')
    print(f"\n✅ 엑셀 파일로 저장 완료: {filename}")

    # DataFrame 생성 (링크 없는 버전)
    df_2 = pd.DataFrame(data)
    df_2 = df_2.drop(columns=["상세페이지링크", "PDF 링크"])


✅ 엑셀 파일로 저장 완료: 카카오_2023-07-01~2024-07-31_리포트 수집.xlsx


# 실제 주가 불러오기

In [5]:
from pykrx import stock
from datetime import timedelta

# 작성일 → datetime 변환
df_2["작성일_dt"] = pd.to_datetime(df_2["작성일"], format="%y.%m.%d")

# 1년 뒤 날짜 계산
df_2["1년뒤_예측일"] = df_2["작성일_dt"] + pd.DateOffset(years=1)

# 종목코드 다시 확인
item_code = ticker_name_map[input_name]

# 1년 뒤 종가 저장용 리스트
actual_prices = []

for idx, row in df_2.iterrows():
    predict_day = row["1년뒤_예측일"]
    from_date = predict_day.strftime("%Y%m%d")
    to_date = (predict_day + timedelta(days=5)).strftime("%Y%m%d")  # 영업일이 아닐 경우 대비
    
    try:
        df_price = stock.get_market_ohlcv_by_date(from_date, to_date, item_code)
        df_price = df_price.reset_index()
        
        # 가장 가까운 날짜의 종가 선택
        if not df_price.empty:
            price = df_price.iloc[0]["종가"]
        else:
            price = None
    except:
        price = None

    actual_prices.append(price)
    time.sleep(0.2)  # 과도한 요청 방지

df_2["1년뒤_실제종가"] = actual_prices


In [6]:
df_2

,제목,증권사,작성일,목표가,투자의견,작성일_dt,1년뒤_예측일,1년뒤_실제종가
0,카카오 주요 자회사 매각 가능성 점검,하이투자증권,24.07.29,"53,000",Buy,2024-07-29,2025-07-29,56000
1,저점을 지나가는 구간,IBK투자증권,24.07.19,"65,000",매수,2024-07-19,2025-07-19,56900
2,무거워진 어깨,SK증권,24.07.16,"62,000",매수,2024-07-16,2025-07-16,56100
3,견조한 톡비즈 vs 제한적인 컨텐츠 성장,교보증권,24.07.11,"60,000",Buy,2024-07-11,2025-07-11,60000
4,반등의 실마리,하나증권,24.07.09,"60,000",Buy,2024-07-09,2025-07-09,60500
...,...,...,...,...,...,...,...,...
71,체질 개선의 기간,삼성증권,23.07.17,"64,000",Buy,2023-07-17,2024-07-17,41200
72,투자 확대 사이클을 견뎌내야 하는 시기,한화투자증권,23.07.14,"75,000",Buy,2023-07-14,2024-07-14,41900
73,매출과 비용의 동반 성장,유진투자증권,23.07.13,"74,000",Buy,2023-07-13,2024-07-13,41900
74,하반기 톡비즈 성장에 달린 주가 회복,SK증권,23.07.11,"78,000",매수,2023-07-11,2024-07-11,42350


# 오차율, 오차율 절댓값 구하기

In [7]:
# 1. 목표가 문자열 → 수치형으로 변환
def parse_price(price_text):
    if not price_text:
        return None
    try:
        return int(price_text.replace(",", "").replace("원", "").strip())
    except:
        return None

df_2["목표가(수치)"] = df_2["목표가"].apply(parse_price)

# 2. 오차율 계산: (목표가 - 실제종가) / 실제종가 * 100
def calculate_error(target, actual):
    try:
        if pd.notnull(target) and pd.notnull(actual) and actual != 0:
            return round((target - actual) / actual * 100, 2)
        else:
            return None
    except:
        return None

df_2["오차율(%)"] = df_2.apply(lambda row: calculate_error(row["목표가(수치)"], row["1년뒤_실제종가"]), axis=1)

df_2

# 절댓값 기준 오차율 낮은 순서로 정렬
df_2["오차율_절댓값"] = df_2["오차율(%)"].abs()
df_2 = df_2.sort_values("오차율_절댓값", ascending=True)

df_2

,제목,증권사,작성일,목표가,투자의견,작성일_dt,1년뒤_예측일,1년뒤_실제종가,목표가(수치),오차율(%),오차율_절댓값
3,견조한 톡비즈 vs 제한적인 컨텐츠 성장,교보증권,24.07.11,"60,000",Buy,2024-07-11,2025-07-11,60000,60000,0.00,0.00
4,반등의 실마리,하나증권,24.07.09,"60,000",Buy,2024-07-09,2025-07-09,60500,60000,-0.83,0.83
5,새로운 성장 전략 제시 필요,한화투자증권,24.07.04,"60,000",Buy,2024-07-04,2025-07-04,58400,60000,2.74,2.74
6,컨텐츠 자회사들의 실적 부진 지속,대신증권,24.07.04,"60,000",Buy,2024-07-04,2025-07-04,58400,60000,2.74,2.74
0,카카오 주요 자회사 매각 가능성 점검,하이투자증권,24.07.29,"53,000",Buy,2024-07-29,2025-07-29,56000,53000,-5.36,5.36
...,...,...,...,...,...,...,...,...,...,...,...
25,탑라인 성장과 비용 컨트롤 효과 극대화,메리츠증권,24.02.19,"83,000",Buy,2024-02-19,2025-02-19,39300,83000,111.20,111.20
65,"많아진 식구, 무거워진 어깨",SK증권,23.08.04,"78,000",매수,2023-08-04,2024-08-04,36300,78000,114.88,114.88
58,2023년 애널리스트 데이 Key Takeaways,이베스트증권,23.09.20,"75,000",Buy,2023-09-20,2024-09-20,34900,75000,114.90,114.90
42,2월에도 풍부한 모멘텀 보유,미래에셋증권,24.01.09,"80,000",매수,2024-01-09,2025-01-09,37200,80000,115.05,115.05


In [9]:
# 1. NaN 제거
df_valid = df_2[df_2["오차율(%)"].notnull()]

# 2. 증권사별 통계 집계
broker_error = (
    df_valid.groupby("증권사")["오차율(%)"]
    .agg([
        ("리포트_수", "count"),
        ("평균_오차율(%)", "mean"),
        ("평균_절댓값_오차율(%)", lambda x: x.abs().mean())
    ])
    .reset_index()
)

# 3. 리포트 수 기준 필터링 (선택 사항)
broker_error = broker_error[broker_error["리포트_수"] >= 1]

# 4. 절댓값 오차율 기준 정렬
broker_error_sorted = broker_error.sort_values("평균_절댓값_오차율(%)")

# 5. 출력
display(broker_error_sorted)


,증권사,리포트_수,평균_오차율(%),평균_절댓값_오차율(%)
8,신한투자증권,3,42.133333,42.133333
13,하나증권,2,44.075000,44.905000
14,하이투자증권,2,43.350000,48.710000
1,IBK투자증권,3,59.240000,59.240000
7,삼성증권,2,64.995000,64.995000
12,키움증권,5,68.470000,68.470000
4,대신증권,7,69.042857,69.042857
15,한화투자증권,7,73.364286,73.364286
3,교보증권,9,75.234444,75.234444
2,SK증권,9,84.594444,84.594444


In [10]:
# 1) NaN 제거 + 날짜형
df_valid = df_2[df_2["오차율(%)"].notnull()].copy()
df_valid["작성일_dt"] = pd.to_datetime(df_valid["작성일_dt"])

# 2) 증권사별 EWM(날짜 가중) 계산
HL_DAYS = 90  # 반감기 90일

def calc_ewm(g):
    g = g.sort_values("작성일_dt").copy()
    g["오차율_EWM"] = g["오차율(%)"].abs().ewm(
        halflife=pd.Timedelta(days=HL_DAYS),
        times=g["작성일_dt"],
        adjust=True   # ← ★ 여기! times를 쓰면 adjust=True 이어야 함
    ).mean()
    return g

df_ewm = df_valid.groupby("증권사", group_keys=False).apply(calc_ewm)

# 3) 증권사별 최종 요약(가장 최신 EWM을 대표값으로)
broker_error = (
    df_ewm.sort_values("작성일_dt")
          .groupby("증권사")
          .agg(
              리포트_수=("오차율(%)", "count"),
              EWM_오차율=("오차율_EWM", "last")
          )
          .sort_values("EWM_오차율")
          .reset_index()
)

display(broker_error)



C:\Users\chloe\AppData\Local\Temp\ipykernel_34904\93774328.py:17: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ewm = df_valid.groupby("증권사", group_keys=False).apply(calc_ewm)


,증권사,리포트_수,EWM_오차율
0,하이투자증권,2,24.472683
1,하나증권,2,34.898936
2,신한투자증권,3,38.816382
3,IBK투자증권,3,39.815149
4,키움증권,5,41.777945
5,교보증권,9,53.449441
6,대신증권,7,53.493482
7,한화투자증권,7,59.730883
8,SK증권,9,66.735803
9,삼성증권,2,67.397352
